In [277]:
import numpy as np
import sys
import plotly.graph_objects as go
import matplotlib.pyplot as plt

import networkx as nx



def init_array(size):
    A = np.random.randint(100, size=(size, size))
    np.fill_diagonal(A, 0)
    A = np.triu(A) + np.tril(A.T)
    return A


def TSP_ph1(Graph, S, destination):
    if (len(S) <= 0):
        #s=list(S)
        return list([(1, destination)]), Graph[0, destination-1]
    else:
        min_dist = sys.maxsize
        min_way = list()
        S=S.difference({destination})
        for _, node in enumerate(S):
            w, d = TSP_ph1(Graph, S.difference({node}), node)
            d += Graph[node - 1, destination - 1]
            if d < min_dist:
                min_dist = d
                min_way = w
                min_way.append((node, destination))
        return min_way, min_dist

def TSP(Graph):
    nodecount=len(Graph[0])
    S = set(range(2, nodecount+1))
    min_dist = sys.maxsize
    min_way = list()
    for _,node in enumerate(S):
        w, d = TSP_ph1(Graph, S, node)
        d += Graph[node - 1, 0]
        if d < min_dist:
            min_dist = d
            min_way = w
            min_way.append((node, 1))
    return min_way, min_dist


In [278]:
import plotly.graph_objects as go

import networkx as nx

def plot_TSP(Graph,way):
    G = nx.from_numpy_array(Graph)
    G.edges(data=True)
    pos = nx.circular_layout(G)
    print(G)

    edge_label_trace = go.Scatter(
        x=[],
        y=[],
        text=[],
        textposition='middle center',
        mode='markers+text',
        hoverinfo='none',
        marker=go.scatter.Marker(
            opacity=0
        ),
        textfont=dict(size=9, color='blue')
    )


    edge_x = []
    edge_y = []
    for edge in G.edges():
        x0, y0 = pos[edge[0]]
        x1, y1 = pos[edge[1]]
        edge_x.append(x0)
        edge_x.append(x1)
        edge_x.append(None)
        edge_y.append(y0)
        edge_y.append(y1)
        edge_y.append(None)

        #invisible node at middle of edges to place edge labels
        edge_label_trace['x']+=((x0+x1)/2,)
        edge_label_trace['y']+=((y0+y1)/2,)
        edge_label_trace['text']+=(Graph[edge[0]][edge[1]],)

    edge_trace = go.Scatter(
        x=edge_x, y=edge_y,
        line=dict(width=0.5, color='#888'),
        hoverinfo='none',
        mode='lines')

    way_x = []
    way_y = []
    arrows = []
    for w in way:
        x0, y0 = pos[w[0]-1]
        x1, y1 = pos[w[1]-1]
        way_x.append(x0)
        way_x.append(x1)
        way_x.append(None)
        way_y.append(y0)
        way_y.append(y1)
        way_y.append(None)
        arrows.append( go.layout.Annotation(
                    x=x1,
                    y=y1,
                    xref="x",
                    yref="y",
                    text=None,
                    showarrow=True,
                    arrowhead=7,
                    ax=x0-x1,
                    ay=y0-y1
                ))    

    way_trace = go.Scatter(
        x=way_x, y=way_y,
        line=dict(width=2, color='#F00'),
        hoverinfo='none',
        mode='lines')


    node_x = []
    node_y = []
    for node in G.nodes():
        x, y = pos[node]
        node_x.append(x)
        node_y.append(y)

    node_trace = go.Scatter(
        x=node_x, y=node_y,
        mode='markers+text',
        hoverinfo='none',
        textposition='middle center',
        marker=dict(
            showscale=False,
            # colorscale options
            #'Greys' | 'YlGnBu' | 'Greens' | 'YlOrRd' | 'Bluered' | 'RdBu' |
            #'Reds' | 'Blues' | 'Picnic' | 'Rainbow' | 'Portland' | 'Jet' |
            #'Hot' | 'Blackbody' | 'Earth' | 'Electric' | 'Viridis' |
            colorscale='Earth',
            reversescale=True,
            color='#FFF',
            size=25,
            line_width=2))
    node_trace.text = [x+1 for x in list(G.nodes())]



    fig = go.Figure(data=[edge_trace, node_trace,edge_label_trace,way_trace],
                 layout=go.Layout(
                    title='Traveling Salesman exact Solution',
                    titlefont_size=16,
                    showlegend=False,
                    hovermode='closest',
                    margin=dict(b=20,l=5,r=5,t=40),
                    annotations=[ dict(
                        text=("Traveling Salesman result : %s, Length=%d" % (way,way_len)),
                        showarrow=False,
                        xref="paper", yref="paper",
                        x=0.005, y=-0.002 ) ].extend(arrows),
                    xaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
                    yaxis=dict(showgrid=False, zeroline=False, showticklabels=False))
                    )
    fig.show()

In [242]:
G1 = np.array([[ 0, 43, 18, 34, 67, 41, 32,  1, 84],
       [43,  0, 93, 51,  1, 86, 44, 92, 27],
       [18, 93,  0, 49, 33, 87, 36, 22,  2],
       [34, 51, 49,  0, 18,  9, 24, 19, 28],
       [67,  1, 33, 18,  0, 89, 85, 64, 57],
       [41, 86, 87,  9, 89,  0, 27,  9,  5],
       [32, 44, 36, 24, 85, 27,  0, 42, 10],
       [ 1, 92, 22, 19, 64,  9, 42,  0, 44],
       [84, 27,  2, 28, 57,  5, 10, 44,  0]])

In [256]:
G1=init_array(9)
G2=init_array(10)
G3=init_array(11)


In [258]:

optimum={
    'G1':dict(zip(['way','cost'],TSP(G1))),
    'G2':dict(zip(['way','cost'],TSP(G2))),
    'G3':dict(zip(['way','cost'],TSP(G3))),
    
}

In [259]:
optimum

{'G1': {'way': [(1, 9),
   (9, 6),
   (6, 8),
   (8, 4),
   (4, 3),
   (3, 2),
   (2, 7),
   (7, 5),
   (5, 1)],
  'cost': 190},
 'G2': {'way': [(1, 10),
   (10, 5),
   (5, 6),
   (6, 2),
   (2, 7),
   (7, 9),
   (9, 8),
   (8, 3),
   (3, 4),
   (4, 1)],
  'cost': 166},
 'G3': {'way': [(1, 9),
   (9, 10),
   (10, 4),
   (4, 3),
   (3, 11),
   (11, 2),
   (2, 6),
   (6, 5),
   (5, 8),
   (8, 7),
   (7, 1)],
  'cost': 261}}

# Optimal Solutions

In [261]:
plot_TSP(G1,optimum['G1']['way'])

In [267]:
plot_TSP(G2,optimum['G2']['way'])

In [268]:
plot_TSP(G3,optimum['G3']['way'])

In [129]:
class a:
    kaka="ba"
    def __init__(self):
        self.pipi="berk"
        
v1=a()
a.kaka="gg"
v2=a()

print(v1.kaka)
print(v2.kaka)
print (a.kaka)

gg
gg
gg


In [281]:
a=np.zeros(G1.shape)+0.001
print(list(range(0,9))/2)

TypeError: unsupported operand type(s) for /: 'list' and 'int'

In [191]:
print(np.array([1,2])==np.array([1,1]))

[ True False]


In [198]:
x= 1 if 0==1 else 2

In [200]:
G1=init_array(9)

In [232]:
import plotly.graph_objects as go

fig = go.Figure()

fig.add_trace(go.Scatter(
    x=[0, 1, 2, 3, 4, 5, 6, 7, 8],
    y=[0, 1, 3, 2, 4, 3, 4, 6, 5]
))


fig.add_trace(go.Scatter(
    x=[0, 1, 2, 3, 4, 5, 6, 7, 8],
    y=[0, 4, 5, 1, 2, 2, 3, 4, 2]
))

fig.update_layout(
    showlegend=False,
    annotations=[
        go.layout.Annotation(
            x=2,
            y=5,
            xref="x",
            yref="y",
            text="dict Text",
            showarrow=True,
            arrowhead=7,
            ax=0,
            ay=-40
        ),
        go.layout.Annotation(
            x=4,
            y=5,
            xref="x",
            yref="y",
            #text="dict Text 2",
            showarrow=True,
            arrowhead=3,
            ax=30,
            ay=-20
        )
    ]
)

fig.show()